# MKIV location analysis
Using the MKIV file to find the angle and the distance position on the Polar data

In [3]:
from scipy.io import loadmat
import pandas as pd 
import numpy as np
from datetime import datetime
from datetime import timedelta

In [60]:
def windows_timestamp_to_python_datetime(x):
    return datetime(1601,1,1) + timedelta(microseconds=x/10)

def creating_a_mkiv_df(mdata, mcolumn = ['time','x','y']):
    mat_dataframe = pd.DataFrame(mdata, columns=mcolumn)
    
    return making_datatime_index_for_MKIV(mat_dataframe) 

def making_datatime_index_for_MKIV(df):
    # Getting the time column from the data frame 
    time_field = df['time']
    
    # Converting the window's time stamp to Python datetime one by one and appending it to a list 
    windows_timestamp = []
    for x in time_field:
        windows_timestamp.append(windows_timestamp_to_python_datetime(x)) 
        
    # Creating a new dataframe with the x and y column and making Python datetime as the index 
    x_and_y_df = mat_dataframe[['x','y']] 
    x_and_y_df.index=windows_timestamp
    
    return x_and_y_df



def finding_min_and_max_for_xy(df):
    # Getting columns and rows where index is higher or equal to 2016-12-29 and lower than 2016-12-31 
    max_and_min_xy_df = df[ ((df.index >= "2016-12-29 00:00:00.000000") & 
                                     (df.index < "2016-12-31 00:00:00.000000") ) ]
    # Getting the max and min value for x and y 
    x_min_and_max = [max_and_min_xy_df['x'].min(), max_and_min_xy_df['x'].max()]
    y_min_and_max = [max_and_min_xy_df['y'].min(), max_and_min_xy_df['y'].max()]
    
    return identifying_polar_data_subset_position(x_min_and_max, y_min_and_max)


def identifying_polar_data_subset_position(x_min_and_max, y_min_and_max):
    
    x_min, x_max = x_min_and_max[0], x_min_and_max[1]
    y_min, y_max = y_min_and_max[0], y_min_and_max[1]
    
    x_difference = x_max - x_min
    y_difference = y_max - y_min
    
    x_start_position = ((256 - x_difference) / 2) +  x_max
    x_end_position =  x_min - ((256 - x_difference) / 2) 
    
    y_start_position = ((256 - y_difference) / 2) +  y_max
    y_end_position = y_min - ((256 - y_difference) / 2) 
    
    start_x_and_y = [x_start_position,y_start_position]
    end_x_and_y  = [x_end_position, y_end_position]
    
    return [start_x_and_y, end_x_and_y]


In [25]:
mat_fn = r"mat/PulseBinM4Dec.mat"
mat = loadmat(mat_fn)
mdata = mat['PulseBinM4']

x_and_y_df = creating_a_mkiv_df(mdata)

In [54]:
# Getting columns and rows where index is higher or equal to 2016-12-29 and lower than 2016-12-31 
max_and_min_xy_df = x_and_y_df[ ((x_and_y_df.index >= "2016-12-29 00:00:00.000000") & (x_and_y_df.index < "2016-12-31 00:00:00.000000") ) ]
# Getting the max and min value for x and y 
x_min_and_max = [max_and_min_xy_df['x'].min(), max_and_min_xy_df['x'].max()]
y_min_and_max = [max_and_min_xy_df['y'].min(), max_and_min_xy_df['y'].max()]

y_min_and_max

[324.0, 362.0]

In [61]:
start_and_end_postion = finding_min_and_max_for_xy(x_and_y_df)

In [56]:
start_and_end_postion

[[3525.5, 471.0], [3269.5, 215.0]]